In [ ]:
import math
import cmath

In [ ]:
class MatrixLogger:
    def __init__(self):
        self.logs = []
        self.step = 0
    def log(self, action, data):
        self.logs.append({'step': self.step, 'action': action, 'data': data})
        self.step += 1
    def clear(self):
        self.logs = []
        self.step = 0
    def print_table(self):
        print("{:<10} {:<30} {:<50}".format("Step", "Action", "Data"))
        print("-" * 90)
        for entry in self.logs:
            print("{:<10} {:<30} {:<50}".format(entry['step'], entry['action'], str(entry['data'])))
    def get_logs(self):
        return self.logs

# Matrix Computations

## Basic Operations

In [ ]:
def matrix_add(A, B):
    return [[A[i][j] + B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def matrix_sub(A, B):
    return [[A[i][j] - B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def split_matrix(matrix):
    n = len(matrix)
    mid = n // 2
    A11 = [row[:mid] for row in matrix[:mid]]
    A12 = [row[mid:] for row in matrix[:mid]]
    A21 = [row[:mid] for row in matrix[mid:]]
    A22 = [row[mid:] for row in matrix[mid:]]
    return A11, A12, A21, A22

def join_quadrants(C11, C12, C21, C22):
    top = [C11[i] + C12[i] for i in range(len(C11))]
    bottom = [C21[i] + C22[i] for i in range(len(C21))]
    return top + bottom

## Strassen's Algorithm

In [ ]:
def strassen(A, B, logger=None):
    n = len(A)
    if n == 1:
        result = [[A[0][0] * B[0][0]]]
        if logger:
            logger.log("Base Case", f"Multiplying {A[0][0]} and {B[0][0]} = {result[0][0]}")
        return result

    A11, A12, A21, A22 = split_matrix(A)
    B11, B12, B21, B22 = split_matrix(B)
    if logger:
        logger.log("Split", f"A-> {A11}, {A12}, {A21}, {A22}; B-> {B11}, {B12}, {B21}, {B22}")

    M1 = strassen(matrix_add(A11, A22), matrix_add(B11, B22), logger)
    if logger: logger.log("M1", f"M1 = {M1}")
    M2 = strassen(matrix_add(A21, A22), B11, logger)
    if logger: logger.log("M2", f"M2 = {M2}")
    M3 = strassen(A11, matrix_sub(B12, B22), logger)
    if logger: logger.log("M3", f"M3 = {M3}")
    M4 = strassen(A22, matrix_sub(B21, B11), logger)
    if logger: logger.log("M4", f"M4 = {M4}")
    M5 = strassen(matrix_add(A11, A12), B22, logger)
    if logger: logger.log("M5", f"M5 = {M5}")
    M6 = strassen(matrix_sub(A21, A11), matrix_add(B11, B12), logger)
    if logger: logger.log("M6", f"M6 = {M6}")
    M7 = strassen(matrix_sub(A12, A22), matrix_add(B21, B22), logger)
    if logger: logger.log("M7", f"M7 = {M7}")

    C11 = matrix_add(matrix_sub(matrix_add(M1, M4), M5), M7)
    C12 = matrix_add(M3, M5)
    C21 = matrix_add(M2, M4)
    C22 = matrix_add(matrix_sub(matrix_add(M1, M3), M2), M6)
    if logger:
        logger.log("Combine Quadrants", f"C11 = {C11}, C12 = {C12}, C21 = {C21}, C22 = {C22}")

    C = join_quadrants(C11, C12, C21, C22)
    if logger:
        logger.log("Result", f"Result = {C}")
    return C

In [ ]:
A = [
    [1, 2],
    [3, 4]
]

B = [
    [5, 6],
    [7, 8]
]

logger = MatrixLogger()

result = strassen(A, B, logger)

print("Matrix A:")
for row in A:
    print(row)
print("\nMatrix B:")
for row in B:
    print(row)
print("\nStrassen's Multiplication Result (A x B):")
for row in result:
    print(row)

print("\nLogger Details:")
logger.print_table()

Matrix A:
[1, 2]
[3, 4]

Matrix B:
[5, 6]
[7, 8]

Strassen's Multiplication Result (A x B):
[19, 22]
[43, 50]

Logger Details:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          Split                          A-> [[1]], [[2]], [[3]], [[4]]; B-> [[5]], [[6]], [[7]], [[8]]
1          Base Case                      Multiplying 5 and 13 = 65                         
2          M1                             M1 = [[65]]                                       
3          Base Case                      Multiplying 7 and 5 = 35                          
4          M2                             M2 = [[35]]                                       
5          Base Case                      Multiplying 1 and -2 = -2                         
6          M3                             M3 = [[-2]]                                       
7          Base Case      

## Matrix Inversion

In [ ]:
def invert_matrix(A, logger=None):
    n = len(A)
    aug = [row[:] + [1 if i == j else 0 for j in range(n)] for i, row in enumerate(A)]
    if logger:
        logger.log("Init", f"Augmented matrix: {aug}")

    for i in range(n):
        pivot = aug[i][i]
        if abs(pivot) < 1e-12:
            for j in range(i+1, n):
                if abs(aug[j][i]) > 1e-12:
                    aug[i], aug[j] = aug[j], aug[i]
                    if logger:
                        logger.log("Swap", f"Swapped rows {i} and {j}")
                    pivot = aug[i][i]
                    break
        if abs(pivot) < 1e-12:
            raise ValueError("Matrix is singular and cannot be inverted.")
        for j in range(2 * n):
            aug[i][j] /= pivot
        if logger:
            logger.log("Normalize", f"Row {i} normalized, new row: {aug[i]}")
        for k in range(n):
            if k != i:
                factor = aug[k][i]
                for j in range(2 * n):
                    aug[k][j] -= factor * aug[i][j]
                if logger:
                    logger.log("Eliminate", f"Eliminated entry at row {k}, column {i}, new row: {aug[k]}")

    inverse = [row[n:] for row in aug]
    if logger:
        logger.log("Result", f"Inverse matrix: {inverse}")
    return inverse

In [ ]:
A = [
    [4, 7, 2],
    [3, 6, 1],
    [2, 5, 1]
]

logger = MatrixLogger()
inv_A = invert_matrix(A, logger)
print("Matrix A:")
for row in A:
    print(row)
print("\nInverse of Matrix A:")
for row in inv_A:
    print(row)
print("\nLogger Details:")
logger.print_table()

Matrix A:
[4, 7, 2]
[3, 6, 1]
[2, 5, 1]

Inverse of Matrix A:
[0.3333333333333335, 1.0, -1.6666666666666665]
[-0.33333333333333337, 0.0, 0.6666666666666666]
[1.0, -2.0, 1.0]

Logger Details:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          Init                           Augmented matrix: [[4, 7, 2, 1, 0, 0], [3, 6, 1, 0, 1, 0], [2, 5, 1, 0, 0, 1]]
1          Normalize                      Row 0 normalized, new row: [1.0, 1.75, 0.5, 0.25, 0.0, 0.0]
2          Eliminate                      Eliminated entry at row 1, column 0, new row: [0.0, 0.75, -0.5, -0.75, 1.0, 0.0]
3          Eliminate                      Eliminated entry at row 2, column 0, new row: [0.0, 1.5, 0.0, -0.5, 0.0, 1.0]
4          Normalize                      Row 1 normalized, new row: [0.0, 1.0, -0.6666666666666666, -1.0, 1.3333333333333333, 0.0]
5          Eliminate       

In [ ]:
def identity(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

def transpose(A):
    m, n = len(A), len(A[0])
    return [[A[i][j] for i in range(m)] for j in range(n)]

def mat_mult(A, B):
    m, p, n = len(A), len(B), len(B[0])
    result = [[0 for _ in range(n)] for _ in range(m)]
    for i in range(m):
        for j in range(n):
            for k in range(p):
                result[i][j] += A[i][k] * B[k][j]
    return result

## Eigenvalues Computation

In [ ]:
def qr_decomposition(A, logger=None):
    m, n = len(A), len(A[0])
    Q = [[0.0 for _ in range(n)] for _ in range(m)]
    R = [[0.0 for _ in range(n)] for _ in range(n)]
    for j in range(n):
        v = [A[i][j] for i in range(m)]
        for k in range(j):
            R[k][j] = sum(Q[i][k] * A[i][j] for i in range(m))
            v = [v[i] - R[k][j] * Q[i][k] for i in range(m)]
        norm_v = math.sqrt(sum(x * x for x in v))
        R[j][j] = norm_v
        if norm_v < 1e-12:  # Handle near-zero column.
            continue
        for i in range(m):
            Q[i][j] = v[i] / norm_v
        if logger:
            logger.log("QR Decomp", f"Column {j} processed; norm = {norm_v}")
    return Q, R

In [ ]:
def qr_algorithm_eig(A, tol=1e-10, max_iter=1000, logger=None):
    n = len(A)
    Ak = [row[:] for row in A]  # Copy of A
    Q_acc = identity(n)
    for it in range(max_iter):
        Q, R = qr_decomposition(Ak, logger)
        Ak_new = mat_mult(R, Q)
        Q_acc = mat_mult(Q_acc, Q)
        off_diag = 0.0
        for i in range(n):
            for j in range(n):
                if i != j:
                    off_diag += abs(Ak_new[i][j])
        if logger:
            logger.log("QR Iteration", f"Iteration {it}: off-diag sum = {off_diag}")
        if off_diag < tol:
            break
        Ak = Ak_new
    eigenvalues = [Ak[i][i] for i in range(n)]
    return eigenvalues, Q_acc

In [ ]:
def svd(A, tol=1e-10, max_iter=1000, logger=None):
    m, n = len(A), len(A[0])
    At = transpose(A)
    AtA = mat_mult(At, A)
    eigenvalues, V = qr_algorithm_eig(AtA, tol, max_iter, logger)
    sigma = [math.sqrt(e) if e > 0 else 0.0 for e in eigenvalues]
    U = [[0.0 for _ in range(n)] for _ in range(m)]
    for i in range(n):
        if sigma[i] > tol:
            v_i = [V[j][i] for j in range(n)]
            Av = [sum(A[row][k] * v_i[k] for k in range(n)) for row in range(m)]
            for row in range(m):
                U[row][i] = Av[row] / sigma[i]
            if logger:
                logger.log("SVD", f"Computed U column {i} from singular value {sigma[i]}")
        else:
            for row in range(m):
                U[row][i] = 0.0
            if logger:
                logger.log("SVD", f"Singular value {i} is near zero; U column set to zero")
    return U, sigma, V

In [ ]:
logger_qr = MatrixLogger()
A_qr = [
    [2, 1, 0],
    [1, 2, 1],
    [0, 1, 2]
]
eigs, evecs = qr_algorithm_eig(A_qr, tol=1e-8, max_iter=100, logger=logger_qr)
print("QR Algorithm Eigenvalue Decomposition")
print("Matrix A:")
for row in A_qr:
    print(row)
print("\nEigenvalues:")
print(eigs)
print("\nEigenvectors (columns of Q_acc):")
for row in evecs:
    print(row)
print("\nQR Logger Details:")
logger_qr.print_table()

logger_svd = MatrixLogger()
A_svd = [
    [1, 0],
    [0, 1],
    [1, 1]
]
U, sigma, V = svd(A_svd, tol=1e-8, max_iter=100, logger=logger_svd)
print("\nSVD of Matrix A")
print("Matrix A:")
for row in A_svd:
    print(row)
print("\nLeft Singular Vectors U (m x n):")
for row in U:
    print(row)
print("\nSingular Values (σ):")
print(sigma)
print("\nRight Singular Vectors V (n x n):")
for row in V:
    print(row)
print("\nSVD Logger Details:")
logger_svd.print_table()

QR Algorithm Eigenvalue Decomposition
Matrix A:
[2, 1, 0]
[1, 2, 1]
[0, 1, 2]

Eigenvalues:
[3.414213562373095, 2.0000000000000004, 0.5857864376269051]

Eigenvectors (columns of Q_acc):
[0.5000000014931172, -0.7071067801307547, 0.5]
[0.7071067811865476, 1.4931171502845353e-09, -0.7071067811865475]
[0.49999999850688276, 0.7071067822423416, 0.5]

QR Logger Details:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          QR Decomp                      Column 0 processed; norm = 2.23606797749979       
1          QR Decomp                      Column 1 processed; norm = 1.6733200530681511     
2          QR Decomp                      Column 2 processed; norm = 1.0690449676496976     
3          QR Iteration                   Iteration 0: off-diag sum = 2.7744160847094568    
4          QR Decomp                      Column 0 processed; norm = 2.8982753

# Fourier Transforms

In [ ]:
class FourierLogger:
    def __init__(self):
        self.logs = []
        self.step = 0
    def log(self, action, data):
        self.logs.append({'step': self.step, 'action': action, 'data': data})
        self.step += 1
    def clear(self):
        self.logs = []
        self.step = 0
    def print_table(self):
        print("{:<10} {:<30} {:<50}".format("Step", "Action", "Data"))
        print("-" * 90)
        for entry in self.logs:
            print("{:<10} {:<30} {:<50}".format(entry['step'], entry['action'], str(entry['data'])))
    def get_logs(self):
        return self.logs

## Discrete Fourier Transforms

In [ ]:
def principal_nth_root_of_unity(n):
    return cmath.exp(2j * cmath.pi / n)

def dft(a, omega, logger=None):
    n = len(a)
    b = [0] * n
    for i in range(n):
        s = 0
        for j in range(n):
            s += a[j] * (omega ** (i * j))
        b[i] = s
        if logger:
            logger.log("DFT", f"b[{i}] = {s}")
    return b

def inverse_dft(b, omega, logger=None):
    n = len(b)
    a = [0] * n
    inv_n = 1 / n
    for j in range(n):
        s = 0
        for i in range(n):
            s += b[i] * (omega ** (-i * j))
        a[j] = inv_n * s
        if logger:
            logger.log("Inverse DFT", f"a[{j}] = {a[j]}")
    return a

def convolution(a, b, omega, logger=None):
    n = len(a)
    A = dft(a, omega, logger)
    B = dft(b, omega, logger)
    C = [A[i] * B[i] for i in range(n)]
    c = inverse_dft(C, omega, logger)
    return c

In [ ]:
a = [1, 2, 3, 4]
n = len(a)
omega = principal_nth_root_of_unity(n)
logger = FourierLogger()

b = dft(a, omega, logger)
print("DFT Result:")
for val in b:
    print(val)
print("\nDFT Logger:")
logger.print_table()

logger.clear()
a_reconstructed = inverse_dft(b, omega, logger)
print("\nInverse DFT Result:")
for val in a_reconstructed:
    print(val)
print("\nInverse DFT Logger:")
logger.print_table()

# Example usage of convolution
u = [1, 2, 3, 4]
v = [4, 3, 2, 1]
logger.clear()
conv = convolution(u, v, omega, logger)
print("\nConvolution (via DFT) Result:")
for val in conv:
    print(val)
print("\nConvolution Logger:")
logger.print_table()

DFT Result:
(10+0j)
(-2.0000000000000004-1.9999999999999996j)
(-2+9.797174393178826e-16j)
(-1.9999999999999982+2.000000000000001j)

DFT Logger:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          DFT                            b[0] = (10+0j)                                    
1          DFT                            b[1] = (-2.0000000000000004-1.9999999999999996j)  
2          DFT                            b[2] = (-2+9.797174393178826e-16j)                
3          DFT                            b[3] = (-1.9999999999999982+2.000000000000001j)   

Inverse DFT Result:
(1.0000000000000004+5.551115123125783e-16j)
(2+2.7755575615628914e-16j)
(2.9999999999999996+1.1102230246251565e-16j)
(4-2.7755575615628914e-16j)

Inverse DFT Logger:
Step       Action                         Data                                              
--------------------

## Fast Fourier Transforms

In [ ]:
def fft(a, logger=None, depth=0):
    n = len(a)
    if n == 1:
        if logger:
            logger.log(f"Base case (depth {depth})", f"Return {a}")
        return a
    even = fft(a[0::2], logger, depth + 1)
    odd = fft(a[1::2], logger, depth + 1)
    y = [0] * n
    omega_n = cmath.exp(2j * cmath.pi / n)
    for k in range(n // 2):
        t = (omega_n ** k) * odd[k]
        y[k] = even[k] + t
        y[k + n // 2] = even[k] - t
        if logger:
            logger.log(f"Depth {depth}, k = {k}",
                       f"even[{k}] = {even[k]}, odd[{k}] = {odd[k]}, "
                       f"t = {t}, y[{k}] = {y[k]}, y[{k + n//2}] = {y[k + n//2]}")
    if logger:
         logger.log(f"Depth {depth} Combined", f"y = {y}")
    return y

def ifft(a, logger=None):
    n = len(a)
    a_conj = [x.conjugate() for x in a]
    if logger:
        logger.log("IFFT", f"Conjugated input: {a_conj}")
    y = fft(a_conj, logger, depth=0)
    y = [x.conjugate() / n for x in y]
    if logger:
        logger.log("IFFT", f"Final result after conjugation and division: {y}")
    return y

In [ ]:
logger_fft = FourierLogger()
a = [1, 2, 3, 4, 5, 6, 7, 8]  # Input vector (length must be a power of 2)
result_fft = fft(a, logger_fft)
print("FFT Result:")
for val in result_fft:
    print(val)
print("\nFFT Logger Details:")
logger_fft.print_table()

logger_fft.clear()
result_ifft = ifft(result_fft, logger_fft)
print("\nInverse FFT Result (should approximate the original vector):")
for val in result_ifft:
    print(val)
print("\nInverse FFT Logger Details:")
logger_fft.print_table()

FFT Result:
(36+0j)
(-4-9.65685424949238j)
(-4.000000000000001-4j)
(-4.000000000000002-1.6568542494923797j)
(-4+0j)
(-3.9999999999999996+1.6568542494923797j)
(-3.999999999999999+4j)
(-3.999999999999998+9.65685424949238j)

FFT Logger Details:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          Base case (depth 3)            Return [1]                                        
1          Base case (depth 3)            Return [5]                                        
2          Depth 2, k = 0                 even[0] = 1, odd[0] = 5, t = (5+0j), y[0] = (6+0j), y[1] = (-4+0j)
3          Depth 2 Combined               y = [(6+0j), (-4+0j)]                             
4          Base case (depth 3)            Return [3]                                        
5          Base case (depth 3)            Return [7]                                        


In [ ]:
def modinv(a, mod):
    return pow(a, -1, mod)

def fft_mod(a, omega, mod, logger=None, depth=0):
    n = len(a)
    if n == 1:
        if logger:
            logger.log(f"FFT Base (depth {depth})", f"Input: {a}")
        return a[:]
    a_even = fft_mod(a[0::2], (omega*omega) % mod, mod, logger, depth+1)
    a_odd  = fft_mod(a[1::2], (omega*omega) % mod, mod, logger, depth+1)
    y = [0] * n
    omega_k = 1
    for k in range(n//2):
        t = (omega_k * a_odd[k]) % mod
        y[k] = (a_even[k] + t) % mod
        y[k + n//2] = (a_even[k] - t) % mod
        if logger:
            logger.log(f"FFT Depth {depth}", f"k={k}, omega_k={omega_k}, t={t}, y[{k}]={y[k]}, y[{k + n//2}]={y[k + n//2]}")
        omega_k = (omega_k * omega) % mod
    if logger:
        logger.log(f"FFT Combine (depth {depth})", f"y = {y}")
    return y

def ifft_mod(a, omega, mod, logger=None):
    n = len(a)
    # Compute inverse omega = omega_inv
    omega_inv = modinv(omega, mod)
    y = fft_mod(a, omega_inv, mod, logger)
    n_inv = modinv(n, mod)
    return [(val * n_inv) % mod for val in y]

def int_to_poly(x, block_size, b, logger=None):
    base = 1 << block_size
    coeff = []
    for _ in range(b):
        coeff.append(x % base)
        x //= base
    if logger:
        logger.log("int_to_poly", f"Digits: {coeff} (base {base})")
    return coeff

def poly_to_int(coeff, block_size, logger=None):
    base = 1 << block_size
    carry = 0
    for i in range(len(coeff)):
        total = coeff[i] + carry
        coeff[i] = total % base
        carry = total // base
    if logger:
        logger.log("poly_to_int", f"Normalized coeff: {coeff}, final carry: {carry}")
    result = 0
    for i in reversed(range(len(coeff))):
        result = result * base + coeff[i]
    result += carry * (base ** len(coeff))
    return result

def next_power_of_two(x):
    return 1 << (x - 1).bit_length()

def poly_mul_mod(p, q, n_fft, mod, logger=None):
    # Zero-pad to length n_fft
    P = p + [0]*(n_fft - len(p))
    Q = q + [0]*(n_fft - len(q))
    if logger:
        logger.log("poly_mul_mod", f"P padded: {P}")
        logger.log("poly_mul_mod", f"Q padded: {Q}")
    g = 3
    omega = pow(g, (mod-1)//n_fft, mod)
    if logger:
        logger.log("poly_mul_mod", f"Using omega = {omega} (n_fft={n_fft}, mod={mod})")
    FP = fft_mod(P, omega, mod, logger)
    FQ = fft_mod(Q, omega, mod, logger)
    Fprod = [(FP[i] * FQ[i]) % mod for i in range(n_fft)]
    if logger:
        logger.log("poly_mul_mod", f"Pointwise product: {Fprod}")
    prod = ifft_mod(Fprod, omega, mod, logger)
    if logger:
        logger.log("poly_mul_mod", f"Convolution result (raw): {prod}")
    return prod

def schoenhage_strassen(u, v, block_size=4, b=8, mod=65537, logger=None):
    poly_u = int_to_poly(u, block_size, b, logger)
    poly_v = int_to_poly(v, block_size, b, logger)

    n_fft = next_power_of_two(2 * b)
    if logger:
        logger.log("schoenhage_strassen", f"FFT length chosen: {n_fft}")

    prod_poly = poly_mul_mod(poly_u, poly_v, n_fft, mod, logger)
    conv_coeff = prod_poly[:2*b - 1]
    if logger:
        logger.log("schoenhage_strassen", f"Raw convolution coefficients: {conv_coeff}")
    result = poly_to_int(conv_coeff, block_size, logger)
    if logger:
        logger.log("schoenhage_strassen", f"Final integer result: {result}")
    return result

In [ ]:
logger_ss = FourierLogger()

u = 123456789
v = 987654321

result_ss = schoenhage_strassen(u, v, block_size=4, b=8, mod=65537, logger=logger_ss)
print("Integer u:", u)
print("Integer v:", v)
print("Product u * v (Schönhage–Strassen):", result_ss)
print("\nSchönhage–Strassen Logger Details:")
logger_ss.print_table()

Integer u: 123456789
Integer v: 987654321
Product u * v (Schönhage–Strassen): 121932631112635269

Schönhage–Strassen Logger Details:
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          int_to_poly                    Digits: [5, 1, 13, 12, 11, 5, 7, 0] (base 16)     
1          int_to_poly                    Digits: [1, 11, 8, 6, 14, 13, 10, 3] (base 16)    
2          schoenhage_strassen            FFT length chosen: 16                             
3          poly_mul_mod                   P padded: [5, 1, 13, 12, 11, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4          poly_mul_mod                   Q padded: [1, 11, 8, 6, 14, 13, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0]
5          poly_mul_mod                   Using omega = 64 (n_fft=16, mod=65537)            
6          FFT Base (depth 4)             Input: [5]                                        
7        